#  III-Anaysis_solved.py
"""                    TD PART III :
#################################################################
    ANALYSIS AND HYPOTHESIS WHITH A GENOME SCALE METABOLIC MODEL
From the given functions, the information you got about the cobra language,
and the result of previous TD part, try to answer the following questions.
files: practical_metaboli_model.pdf
model : e_coli_core.xml
        other to download
For a better understanding, you can also open the file functions.py to see how some function are built.
During this tutorial you will use object oriented language.

In [1]:
######A
# 1- Load module:
import io
import cobra
import functions
from functions import *

In [2]:
%%capture
#2- Import e_coli_core model:
#cobra.io.save_json_model()
model = cobra.io.read_sbml_model('Data/f_prausnitzii.xml')

In [3]:
#3 - Print information about the model:
info(model)

Metabolites :  833
Reactions :  1030
Genes :  0
Compartments :  {'c': 'Extracellular', 'e': 'Cytosolic'}
Objective function : 
 Maximize
0 



In [4]:
#4- Lauch the flux balance analysis without changing any parameter:
fba = model.optimize()

In [5]:
#5- Print the objective value. Was it what you expected? 
print(fba.objective_value)

0.0


In [6]:
#6- Print fluxes how many reaction are active in this condition?
fluxes = deleteNull(fba.fluxes)
print(fluxes)
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes),len(fluxes)/len(model.reactions)*100))

ACKr                         -1.278113e+00
ACtr                         -1.278113e+00
ADD                           8.719789e-17
ADK1                          7.743212e-16
ADK2                         -1.208726e-15
ADK3                         -8.719789e-17
ADOCBLS                       3.814908e-17
ALAASP1c                      2.945283e-01
ALAASPabc                     2.945283e-01
ALAte                         2.945283e-01
ALTRH                         1.572642e+00
ARGSL                        -1.105849e+00
ARGSSr                       -1.105849e+00
ARGt2r                        1.105849e+00
ASPCT                         1.105849e+00
ASPTA                        -2.383962e+00
ASPt2r                       -2.678491e+00
ATPM                          8.430000e+00
CBLAT                        -3.814908e-17
DADK                         -8.719789e-17
DDGLKr                        1.572642e+00
DHORDfum                      1.105849e+00
DHORTS                       -1.105849e+00
DM_NA1     

In [7]:
#7- From the fluxes, define which metabolites are consume and produce by the model?
#Verify your result with the summary function:
"""
PRODUCE     CONSUME
02          co2
glc         h
nh4         h2o
pi
"""
print(model.summary(),'\n')

Non-linear or non-reaction model objective. Falling back to minimal display.


Objective
nan Expression = nan

Uptake
------
Metabolite                 Reaction  Flux  C-Number  C-Flux
     h2o_e  EX_h2o_LPAREN_e_RPAREN_ 3.143         0   0.00%
     ins_e  EX_ins_LPAREN_e_RPAREN_ 3.143        10  99.52%
    hpyr_c                sink_hpyr  0.05         3   0.48%

Secretion
---------
    Metabolite                    Reaction   Flux  C-Number  C-Flux
         but_e     EX_but_LPAREN_e_RPAREN_ -0.025         4   0.32%
         co2_e     EX_co2_LPAREN_e_RPAREN_  -0.05         1   0.16%
 glyc_DASH_R_e  EX_glyc_R_LPAREN_e_RPAREN_  -0.05         3   0.48%
           h_e       EX_h_LPAREN_e_RPAREN_ -5.212         0   0.00%
        hxan_e    EX_hxan_LPAREN_e_RPAREN_ -3.143         5  49.76%
  lac_DASH_D_e   EX_lac_D_LPAREN_e_RPAREN_ -5.188         3  49.29%
 



In [8]:
#8- print the summary of one of this metabolite to see how it's produced and consumed
print(model.metabolites.nh4_c.summary(),'\n')

nh4_c
=====
Formula: H4N

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: [] 



In [9]:
#9- Verify if the model can grow without oxygene:
## Create a new model to keep initial state

model2 = model.copy()
## Set upper bound and lower bound :
model2.reactions.get_by_id('EX_o2_LPAREN_e_RPAREN_').lower_bound = 0

In [10]:
## Launch fba
fba2 = model2.optimize()
fluxes2 = deleteNull(fba2.fluxes)
print(model2.summary())
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes2),len(fluxes2)/len(model.reactions)*100))
## Conclude:
"""YES"""

Non-linear or non-reaction model objective. Falling back to minimal display.


Objective
nan Expression = nan

Uptake
------
Metabolite                 Reaction  Flux  C-Number  C-Flux
     h2o_e  EX_h2o_LPAREN_e_RPAREN_ 3.143         0   0.00%
     ins_e  EX_ins_LPAREN_e_RPAREN_ 3.142        10  99.52%
    hpyr_c                sink_hpyr  0.05         3   0.48%

Secretion
---------
    Metabolite                    Reaction   Flux  C-Number  C-Flux
         but_e     EX_but_LPAREN_e_RPAREN_ -0.025         4   0.32%
         co2_e     EX_co2_LPAREN_e_RPAREN_  -0.05         1   0.16%
 glyc_DASH_R_e  EX_glyc_R_LPAREN_e_RPAREN_  -0.05         3   0.48%
           h_e       EX_h_LPAREN_e_RPAREN_ -5.213         0   0.00%
        hxan_e    EX_hxan_LPAREN_e_RPAREN_ -3.142         5  49.76%
  lac_DASH_D_e   EX_lac_D_LPAREN_e_RPAREN_ -5.187         3  49.29%


Active reaction : 45 which represent 4.368932% of the model


'YES'

In [11]:
#10- Verify if the model can use other carbon sources: lactate or formate.
## create a copy of the model
model3 = model.copy()
print(model3.exchanges)
## Close the model
model3= closeModel(model3)
## Change bound to allow entry of the target metabolite.
model3.reactions.get_by_id('EX_lac__D_e').lower_bound =-10
for rxn in model3.exchanges:
    print(rxn.lower_bound,'<',rxn.id,'<',rxn.upper_bound)
fba3 = model3.optimize()
fluxes3 = deleteNull(fba3.fluxes)
print(model3.summary())
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes3),len(fluxes3)/len(model.reactions)*100))

[<Reaction EX_3mop_LPAREN_e_RPAREN_ at 0x7f17234d6910>, <Reaction EX_4abz_LPAREN_e_RPAREN_ at 0x7f17234d6950>, <Reaction EX_5oxpro_LPAREN_e_RPAREN_ at 0x7f17234d6990>, <Reaction EX_Lcyst_LPAREN_e_RPAREN_ at 0x7f17234d69d0>, <Reaction EX_Lcystin_LPAREN_e_RPAREN_ at 0x7f17234d6a10>, <Reaction EX_ac_LPAREN_e_RPAREN_ at 0x7f17234d6a50>, <Reaction EX_acasp_LPAREN_e_RPAREN_ at 0x7f17234d6a90>, <Reaction EX_acgal_LPAREN_e_RPAREN_ at 0x7f17234d6ad0>, <Reaction EX_acgalglcur_LPAREN_e_RPAREN_ at 0x7f17234d6b10>, <Reaction EX_acgam_LPAREN_e_RPAREN_ at 0x7f17234d6b50>, <Reaction EX_acglu_LPAREN_e_RPAREN_ at 0x7f17234d6b90>, <Reaction EX_acnam_LPAREN_e_RPAREN_ at 0x7f17234d6bd0>, <Reaction EX_ade_LPAREN_e_RPAREN_ at 0x7f17234d6c10>, <Reaction EX_adocbl_LPAREN_e_RPAREN_ at 0x7f17234d6c50>, <Reaction EX_ala_D_LPAREN_e_RPAREN_ at 0x7f17234d6c90>, <Reaction EX_ala_L_LPAREN_e_RPAREN_ at 0x7f17234d6cd0>, <Reaction EX_alaala_LPAREN_e_RPAREN_ at 0x7f17234d6d10>, <Reaction EX_alaasp_LPAREN_e_RPAREN_ at 0x7f

KeyError: 'EX_lac__D_e'

In [ ]:
#11- It is possible to produce  acetate and to grow? change the objective function (growth min 0.20):
model4=model.copy()
model4.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM').lower_bound= 0.20
model4.reactions.get_by_id('EX_ac_e').lower_bound= 0.1
model4.objective = {model4.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM'): 1, model4.reactions.get_by_id('EX_ac_e') : 1}
util.solver.linear_reaction_coefficients(model4)
print(model4.objective)
print(model4.summary())
print('\nActive reaction : %i which represent %f%% of the model' %(len(fluxes3),len(fluxes3)/len(model.reactions)*100))
#change the coefficient for ac form 1 to 0.5 for example. What do you observe?

In [ ]:
#######B
#Lets look all of this in an entire model;
"""  In biggest model is more difficult to interpret something. Lets see if we can answer to some
question using different functions.
"""

"""  I - You know that you will receive a Saccharomyces cerevisiae strain that you have to grow.
You have 3 different media. Which one will you use to be sure that the strain will grow?"""

# Download one model from bigg database 
model = io.read_sbml_model('Data/iND750.xml')

In [ ]:
# Verify that the model can grow without changing any parameter:

print(model.summary())
fba = model.optimize()

In [ ]:
# Verify how the model can grow in each medium and conclude:
Medium1 = ['ac_e','ade_e','ala__L_e','arg__L_e','asn__L_e','asp__L_e','ca2_e',
           'cit_e','cys_L_e','man_e','gln__L_e','glu__L_e','h2o_e','his__L_e',
           'ile__L_e','mg2_e','nad_e','nh4_e','nac_e','pi_e','pro__L_e','ser__L_e',
           'so4_e','try__L_e','tyr__L_e','trp__L_e','h2_e','o2_e' ] #c = mannose, - uracile pour medium

In [ ]:
model1= medium(model,Medium1)
print('\nMedium1')
model.optimize()
print(model1.summary())
Medium2 = ['ac_e','ade_e','ala__L_e','arg__L_e','asn__L_e','asp__L_e','ca2_e',
           'cit_e','cu2_e','cys_L_e','fe2_e','fe3_e','fru_e','glc__D_e',
           'gln__L_e','glu__L_e','h2o_e','his__L_e','ile__L_e','mg2_e','nad_e',
           'nh4_e','nac_e','no3_e','pi_e','pro__L_e','ser__L_e','so4_e','try__L_e',
           'tyr__L_e','ura_e','val__L_e','trp__L_e','h2_e','o2_e' ]# 2 carbon source less mets

In [ ]:
model1.medium

In [ ]:
model2 = medium(model,Medium2)
print('\nMedium2')
print(model2.summary())
Medium3 = ['4abz_e','ac_e','ace_e','ade_e','ala__L_e','arg__L_e','ascb__L_e','asn__L_e',
           'asp__L_e','btn_e','cbl1_e','cit_e','cl_e','cobalt2_e',
           'cu2_e','cys_L_e','fe2_e','fe3_e','fol_e','glc__D_e','glu__L_e','gly_e','gthrd_e',
           'gua_e','h2o_e','hco3_e','his__L_e','i_e','ile__L_e','inost_e','k_e','leu_L_e','met__L_e','co2_e','no2_e',
           'mg2_e','mn2_e','mndn_e','mobd_e','na1_e','nad_e','nh4_e','nac_e','ni2_e',
           'no3_e','pi_e','pnto__R_e','pro__L_e','pydam_e','pydxn_e','rezrn_e','ribflv_e',
           'ser__L_e','so4_e','thci_e','thm_e','try__L_e','tyr__L_e','ura_e','val__L_e',
           'xan_e','zn2_e','h_e','trp__L_e', 'mobd_e','slnt_e', 'tungs_e','lipoate_e',
           'pydx_e','o2_e' ]# - fructose - gl rich with one carbone source

In [ ]:
model3 = medium(model,Medium3)
print('\nMedium3')
help(cobra.medium)
print(model3.summary())

In [ ]:
""" II- You work for an agribusiness company specialized in yoghurt production.
In, purpose to improve the taste of some product, you want to produce the molecule RR 2 3 Butanediol which gave the butter/ creamy taste.
Your company whant a biological way to produce this molecule instead of chemical.
you have 4 species in your bacteria collection:
    - Saccharomyces cerevisiae
    - Lactococcus lactis
    - Klebsiella pneumoniae
Can you tell which species is the most likely to produce this molecule? """

In [ ]:
 #Find molecule id on Bigg database / its exchange reaction associated
met = 'btd_rr_e'
rxn = 'EX_btd_RR_e'

In [ ]:
# Load the 3 models

model1 = io.read_sbml_model('iMM904.xml')
model2 = io.read_sbml_model('iNF517.xml')
model3 = io.read_sbml_model('iYL1228.xml')

In [ ]:
# Run following function for 'lactococcus lactis':
model2 = updateLactococcus(model2)

In [ ]:
# Verify that models can grow:
print('\nSaccharomyces')
print(model1.summary())
print('\nLactococcus')
print(model2.summary())
print('\nKlebsiella')
print(model3.summary())

In [ ]:
#for each model find biomass function position or name
type1 = reactionsType(model1)
biomPos1 = type1.index('biomass')

type2 = reactionsType(model2)
biomPos2 = type2.index('biomass')

type3 = reactionsType(model3)
biomPos3 = type3.index('biomass')

In [ ]:
# Choice 1 :
#- Update the biomasse bound (>0)
#- Change the objective function
#- Run FBA
print('\nChoice1')
model1.reactions[biomPos1].lower_bound =0.1
model2.reactions[biomPos2].lower_bound =0.1
model3.reactions[biomPos3].lower_bound =0.1

model1.objective = {model1.reactions.get_by_id('EX_btd_RR_e'): 1}
model2.objective = {model2.reactions.get_by_id('EX_btd_RR_e'): 1}
model3.objective = {model3.reactions.get_by_id('EX_btd_RR_e'): 1}

print('\nSaccharomyces')
print(model1.summary())
print('\nLactococcus')
print(model2.summary())
print('\nKlebsiella')
print(model3.summary())

In [ ]:
#Choice 2 :
#- change the objective function with both reactions
#- Run FBA
print('\nChoice2')
model1.objective = {model1.reactions.get_by_id('EX_btd_RR_e'): 1, model1.reactions[biomPos1] : 1}
model2.objective = {model2.reactions.get_by_id('EX_btd_RR_e'): 1, model2.reactions[biomPos2] : 1}
model3.objective = {model3.reactions.get_by_id('EX_btd_RR_e'): 1, model3.reactions[biomPos3] : 1}

print('\nSaccharomyces')
print(model1.summary())
print('\nLactococcus')
print(model2.summary())
print('\nKlebsiella')
print(model3.summary())

In [ ]:
# What can you conclude?
"""Lactobacilus lactis is the best producer of this molecule with th highet flux in the exchange
reaction."""

In [ ]:
# Did you check if models could spontaneously produce the molecule?
""" Lactococcus lactis is producing the molecule when we launch the FBA for growth.
we think that its more likly to produce the molecule"""